# Feature engineering

In [2]:
#Import all neccessary libraries
import os
import bson
import numpy as np
import pandas as pd; pd.set_option('display.max_columns', None)

In [3]:
os.chdir('c:\\Users\\kavou\\OneDrive\\Έγγραφα\\Πρακτική - Μεταπτυχιακό\\MSA\\Courses\\CSE 6748\\LBA project\\Data\\pickle')
os.getcwd()

'c:\\Users\\kavou\\OneDrive\\Έγγραφα\\Πρακτική - Μεταπτυχιακό\\MSA\\Courses\\CSE 6748\\LBA project\\Data\\pickle'

In [4]:
#Import neccessary pickle files
shots = pd.read_pickle(r'Cleaned NCAAM1 21-22 shotsdf.pickle')
player = pd.read_pickle(r'NCAAM1 21-22 playerdictionary.pickle')
segment = pd.DataFrame.from_records(pd.read_pickle(r'NCAAM1 21-22 segment data.pickle'))

In [10]:
segment = segment[['game_id','players_team1','players_team2']]
segment["all_players"] = segment["players_team1"]+segment["players_team2"]
segments_df_long = segment.explode(["all_players"])
segments_df_long = segments_df_long.drop(columns=['players_team1','players_team2'])
segments_df_long = segments_df_long.drop_duplicates()
segments_df_long

,game_id,all_players
0,618bfc21815098ca6660f740,618bf7f3815098ca6660f336
0,618bfc21815098ca6660f740,618bf7f3815098ca6660f338
0,618bfc21815098ca6660f740,618bf7f3815098ca6660f334
0,618bfc21815098ca6660f740,618bf7f2815098ca6660f331
0,618bfc21815098ca6660f740,618bf7f2815098ca6660f332
...,...,...
152283,6256f69516aeeef07951395a,61aa93425e9ccfe1aca8efed
152284,6256f69516aeeef07951395a,61aa69e103fae26ddd299fce
152289,6256f69516aeeef07951395a,61aa93425e9ccfe1aca8eff0
152290,6256f69516aeeef07951395a,61aa69e103fae26ddd299fc8


In [11]:
segments_df_long['played'] = 1
segments_df_long = segments_df_long.drop(columns=['game_id']).groupby(['all_players']).sum().reset_index()
segments_df_long

,all_players,played
0,618bf7f2815098ca6660f330,3
1,618bf7f2815098ca6660f331,24
2,618bf7f2815098ca6660f332,19
3,618bf7f3815098ca6660f333,23
4,618bf7f3815098ca6660f334,24
...,...,...
5006,62565fa22c06a1d6338c6b58,1
5007,62565fff2c06a1d6338c8197,1
5008,6256600f2c06a1d6338c85cd,1
5009,6256f2a116aeeef0794ff950,2


Create a dataframe with all possible combinations of player_id, season_id, team_id

In [12]:
#merge the players with the shots dataframe to add the team_id column 
#that is neccessary for merging with the games dataframe
player_team = pd.merge(player[['player','seasonid']].rename(columns={'seasonid':'season_id','player':'player_id'}),
    shots[['team_id','season_id']].drop_duplicates(), how='left', on='season_id')
player_team.to_pickle("team_player_df.pickle") 
player_team

,player_id,season_id,team_id
0,618da14c4db298feb94be0d1,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3
1,618da14c4db298feb94be0d2,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3
2,618da14c4db298feb94be0d3,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3
3,618da14c4db298feb94be0d4,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3
4,618da14c4db298feb94be0d5,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3
...,...,...,...
4394,6192aaef73c0a0725fd808cc,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb
4395,6255a71bed9cbb2fe0fe5d07,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb
4396,6255a71bed9cbb2fe0fe5d08,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb
4397,6255cd5d2c06a1d633649449,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb


In [13]:
player_team = pd.merge(player_team, segments_df_long.rename(columns={'all_players':'player_id'}), on=['player_id'], how='left')
player_team.played = player_team.played.fillna(0).astype(int)
player_team

,player_id,season_id,team_id,played
0,618da14c4db298feb94be0d1,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3,24
1,618da14c4db298feb94be0d2,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3,24
2,618da14c4db298feb94be0d3,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3,23
3,618da14c4db298feb94be0d4,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3,24
4,618da14c4db298feb94be0d5,5e875e684dc25ebee5af69e6,5e875e684dc25ebee5af69e3,22
...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb,12
4395,6255a71bed9cbb2fe0fe5d07,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb,18
4396,6255a71bed9cbb2fe0fe5d08,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb,7
4397,6255cd5d2c06a1d633649449,5e875e684dc25ebee5af68be,5e875e684dc25ebee5af68bb,2


In [14]:
#Split the Stat column into 2s/3s and Makes/Misses
shots_df = shots.copy()
shots_df.insert(13, 'Type', shots_df.Stat.str[:-4])
shots_df.insert(14, 'Result', shots_df.Stat.str[-4:])
shots_df.Result = np.where(shots_df.Result=='Make', 1, 0)
shots_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Floater,Paint,Jumpshot,Red Zone,Assisted,Layup,Rim,Finish,Pullup,Second Chance,Blocked,Off Turnover,Off Steal,And1,ATO,Turnaround,Step Back,Drive,Hook Shot,Dunk,Fastbreak
0,618bfc21815098ca6660f740,5e875e684dc25ebee5af6a27,5e875e684dc25ebee5af6a2a,618bf7f3815098ca6660f336,618bfbfb815098ca6660f368,0,0,1,618bfbeb815098ca6660f343,19:40,1180,TwoMiss,2-2,Two,0,22.0,10.3,59.5,5.91,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,618bfc21815098ca6660f740,5e875e684dc25ebee5af6a27,5e875e684dc25ebee5af6a2a,618bf7f3815098ca6660f338,618bfbfb815098ca6660f36a,0,0,1,618bfbeb815098ca6660f343,19:0,1140,ThreeMiss,7-1,Three,0,7.0,23.5,45.4,25.63,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,618bfc21815098ca6660f740,5e875e684dc25ebee5af6a27,5e875e684dc25ebee5af6a2a,618bf7f3815098ca6660f336,618bfbfb815098ca6660f36b,3,2,1,618bfbeb815098ca6660f343,18:31,1111,ThreeMake,7-2,Three,1,23.5,32.9,86.9,27.69,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,618bfc21815098ca6660f740,5e875e684dc25ebee5af6a27,5e875e684dc25ebee5af6a2a,618bf7f2815098ca6660f331,618bfbfb815098ca6660f36e,5,5,1,618bfbeb815098ca6660f343,17:59,1079,TwoMake,1,Two,1,24.0,6.6,53.1,1.66,False,True,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,618bfc21815098ca6660f740,5e875e684dc25ebee5af6a27,5e875e684dc25ebee5af6a2a,618bf7f2815098ca6660f332,618bfbfb815098ca6660f373,5,7,1,618bfbeb815098ca6660f343,16:44,1004,TwoMiss,4-2,Two,0,30.0,25.4,103.9,20.74,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522928,6256f69516aeeef07951395a,5e875e684dc25ebee5af69bb,5e875e684dc25ebee5af69be,61aa93425e9ccfe1aca8efef,6256f69416aeeef07951386d,63,71,2,6256f68e16aeeef0795135e2,1:59,119,TwoMiss,1,Two,0,26.5,6.6,138.4,2.00,False,True,False,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
522929,6256f69516aeeef07951395a,5e875e684dc25ebee5af69bb,5e875e684dc25ebee5af69be,61aa93425e9ccfe1aca8efeb,6256f69416aeeef079513874,68,75,2,6256f68e16aeeef0795135e4,0:53,53,ThreeMake,6-2,Three,1,22.0,30.1,83.1,25.01,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
522930,6256f69516aeeef07951395a,5e875e684dc25ebee5af69bb,5e875e684dc25ebee5af69be,61aa93425e9ccfe1aca8efe9,6256f69416aeeef07951387d,71,77,2,6256f68e16aeeef0795135e6,0:26,26,TwoMake,2-2,Two,1,23.0,10.3,68.5,5.47,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
522931,6256f69516aeeef07951395a,5e875e684dc25ebee5af69bb,5e875e684dc25ebee5af69be,61aa93425e9ccfe1aca8efe9,6256f69416aeeef079513881,71,78,2,6256f68e16aeeef0795135e7,0:13,13,TwoMiss,2-2,Two,0,26.0,11.3,99.4,6.11,False,True,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


Feature creation for 2 pointers

In [15]:
#Filter on 2 point shots
Twos_df = shots_df[shots_df.Type=='Two']
two_pointers_df = Twos_df.groupby(['player_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'2PM', 'count':'2PA'})
two_pointers_df

,player_id,2PM,2PA
0,618bf7f2815098ca6660f330,0,5
1,618bf7f2815098ca6660f331,82,122
2,618bf7f2815098ca6660f332,47,92
3,618bf7f3815098ca6660f333,6,16
4,618bf7f3815098ca6660f334,99,218
...,...,...,...
4611,62563e772c06a1d63384b7ea,2,2
4612,625644cb2c06a1d633863180,1,1
4613,62564bb12c06a1d63387f6b5,0,1
4614,62565fa22c06a1d6338c6b58,0,1


In [18]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
two_pointers_df = pd.merge(player_team[['player_id','played']], two_pointers_df, how= 'left', on='player_id').fillna(0)
two_pointers_df

,player_id,played,2PM,2PA
0,618da14c4db298feb94be0d1,24,64.0,141.0
1,618da14c4db298feb94be0d2,24,89.0,170.0
2,618da14c4db298feb94be0d3,23,11.0,21.0
3,618da14c4db298feb94be0d4,24,64.0,121.0
4,618da14c4db298feb94be0d5,22,38.0,92.0
...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,4.0,6.0
4395,6255a71bed9cbb2fe0fe5d07,18,26.0,45.0
4396,6255a71bed9cbb2fe0fe5d08,7,1.0,2.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0


In [19]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Dunk', 'Layup', 'Floater', 'Hook Shot', 'Jumpshot', 'Pullup', 'Turnaround', 'Step Back']:
    temp = Twos_df[Twos_df[stat]==True]
    temp_df = temp.groupby(['player_id']).Result.agg(['sum','count']).reset_index().rename(
        columns={'sum': stat+'s made', 'count': stat+'s attempted'})
    two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)
two_pointers_df

,player_id,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted
0,618da14c4db298feb94be0d1,24,64.0,141.0,1.0,1.0,40.0,78.0,3.0,7.0,1.0,2.0,2.0,18.0,0.0,1.0,5.0,16.0,2.0,2.0
1,618da14c4db298feb94be0d2,24,89.0,170.0,2.0,2.0,56.0,108.0,0.0,1.0,12.0,20.0,2.0,14.0,0.0,0.0,2.0,8.0,0.0,0.0
2,618da14c4db298feb94be0d3,23,11.0,21.0,0.0,0.0,7.0,12.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,618da14c4db298feb94be0d4,24,64.0,121.0,0.0,0.0,29.0,57.0,3.0,4.0,15.0,22.0,11.0,24.0,0.0,0.0,3.0,10.0,0.0,0.0
4,618da14c4db298feb94be0d5,22,38.0,92.0,0.0,0.0,34.0,62.0,0.0,3.0,0.0,2.0,2.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,4.0,6.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4395,6255a71bed9cbb2fe0fe5d07,18,26.0,45.0,0.0,0.0,22.0,34.0,0.0,0.0,1.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4396,6255a71bed9cbb2fe0fe5d08,7,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#Calculated percentage of made 2p jumpshots that were assisted
temp_df = Twos_df[(Twos_df['Jumpshot']==True)&(Twos_df['Result']==1)].groupby(
    ['player_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': 'Jumpshots assisted on'})
temp_df

,player_id,Jumpshots assisted on
0,618bf7f2815098ca6660f331,3
1,618bf7f2815098ca6660f332,3
2,618bf7f3815098ca6660f333,2
3,618bf7f3815098ca6660f334,3
4,618bf7f3815098ca6660f335,0
...,...,...
3433,6255f7a52c06a1d633713153,1
3434,6256030a2c06a1d633748d54,0
3435,625604642c06a1d63374fe1d,0
3436,625620bb2c06a1d6337cae56,0


In [21]:
two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)
two_pointers_df

,player_id,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted,Jumpshots assisted on
0,618da14c4db298feb94be0d1,24,64.0,141.0,1.0,1.0,40.0,78.0,3.0,7.0,1.0,2.0,2.0,18.0,0.0,1.0,5.0,16.0,2.0,2.0,0.0
1,618da14c4db298feb94be0d2,24,89.0,170.0,2.0,2.0,56.0,108.0,0.0,1.0,12.0,20.0,2.0,14.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0
2,618da14c4db298feb94be0d3,23,11.0,21.0,0.0,0.0,7.0,12.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,618da14c4db298feb94be0d4,24,64.0,121.0,0.0,0.0,29.0,57.0,3.0,4.0,15.0,22.0,11.0,24.0,0.0,0.0,3.0,10.0,0.0,0.0,2.0
4,618da14c4db298feb94be0d5,22,38.0,92.0,0.0,0.0,34.0,62.0,0.0,3.0,0.0,2.0,2.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,4.0,6.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4395,6255a71bed9cbb2fe0fe5d07,18,26.0,45.0,0.0,0.0,22.0,34.0,0.0,0.0,1.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4396,6255a71bed9cbb2fe0fe5d08,7,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#2P% per zone for each player
Zones2 = Twos_df.groupby(['player_id','Zone']).Result.agg(['sum','count']).reset_index()
Zones2

,player_id,Zone,sum,count
0,618bf7f2815098ca6660f330,1,0,5
1,618bf7f2815098ca6660f331,1,62,87
2,618bf7f2815098ca6660f331,2-1,1,1
3,618bf7f2815098ca6660f331,2-2,13,19
4,618bf7f2815098ca6660f331,2-3,1,1
...,...,...,...,...
30925,62563e772c06a1d63384b7ea,1,2,2
30926,625644cb2c06a1d633863180,2-1,1,1
30927,62564bb12c06a1d63387f6b5,2-2,0,1
30928,62565fa22c06a1d6338c6b58,2-2,0,1


In [23]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones2.pivot(index=['player_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])#remove leading space
pivot_zones

,player_id,1 made,2-1 made,2-2 made,2-3 made,3-1 made,3-2 made,3-3 made,4-1 made,4-2 made,4-3 made,1 attempted,2-1 attempted,2-2 attempted,2-3 attempted,3-1 attempted,3-2 attempted,3-3 attempted,4-1 attempted,4-2 attempted,4-3 attempted
0,618bf7f2815098ca6660f330,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,618bf7f2815098ca6660f331,62.0,1.0,13.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,87.0,1.0,19.0,1.0,1.0,2.0,1.0,2.0,4.0,4.0
2,618bf7f2815098ca6660f332,26.0,2.0,5.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,42.0,8.0,8.0,2.0,4.0,6.0,6.0,3.0,4.0,9.0
3,618bf7f3815098ca6660f333,3.0,NaN,0.0,NaN,NaN,2.0,NaN,0.0,1.0,0.0,5.0,NaN,1.0,NaN,NaN,3.0,NaN,1.0,3.0,3.0
4,618bf7f3815098ca6660f334,40.0,4.0,12.0,3.0,1.0,13.0,1.0,2.0,22.0,1.0,68.0,8.0,34.0,6.0,7.0,32.0,4.0,4.0,48.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,62563e772c06a1d63384b7ea,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4612,625644cb2c06a1d633863180,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4613,62564bb12c06a1d63387f6b5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4614,62565fa22c06a1d6338c6b58,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
two_pointers_df = pd.merge(two_pointers_df, pivot_zones, how= 'left', on=['player_id']).fillna(0)
two_pointers_df

,player_id,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted,Jumpshots assisted on,1 made,2-1 made,2-2 made,2-3 made,3-1 made,3-2 made,3-3 made,4-1 made,4-2 made,4-3 made,1 attempted,2-1 attempted,2-2 attempted,2-3 attempted,3-1 attempted,3-2 attempted,3-3 attempted,4-1 attempted,4-2 attempted,4-3 attempted
0,618da14c4db298feb94be0d1,24,64.0,141.0,1.0,1.0,40.0,78.0,3.0,7.0,1.0,2.0,2.0,18.0,0.0,1.0,5.0,16.0,2.0,2.0,0.0,44.0,4.0,4.0,2.0,2.0,3.0,1.0,1.0,3.0,0.0,80.0,13.0,15.0,4.0,5.0,11.0,3.0,1.0,9.0,0.0
1,618da14c4db298feb94be0d2,24,89.0,170.0,2.0,2.0,56.0,108.0,0.0,1.0,12.0,20.0,2.0,14.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,71.0,4.0,10.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,116.0,12.0,25.0,7.0,3.0,4.0,1.0,0.0,1.0,1.0
2,618da14c4db298feb94be0d3,23,11.0,21.0,0.0,0.0,7.0,12.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,12.0,2.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0
3,618da14c4db298feb94be0d4,24,64.0,121.0,0.0,0.0,29.0,57.0,3.0,4.0,15.0,22.0,11.0,24.0,0.0,0.0,3.0,10.0,0.0,0.0,2.0,29.0,8.0,13.0,1.0,1.0,4.0,1.0,1.0,6.0,0.0,51.0,12.0,21.0,3.0,6.0,14.0,1.0,1.0,12.0,0.0
4,618da14c4db298feb94be0d5,22,38.0,92.0,0.0,0.0,34.0,62.0,0.0,3.0,0.0,2.0,2.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,2.0,5.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,53.0,5.0,12.0,6.0,1.0,7.0,5.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,4.0,6.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4395,6255a71bed9cbb2fe0fe5d07,18,26.0,45.0,0.0,0.0,22.0,34.0,0.0,0.0,1.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,23.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,3.0,3.0,1.0,2.0,1.0,0.0,1.0,0.0
4396,6255a71bed9cbb2fe0fe5d08,7,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
two_pointers_df.columns[1:].to_list()

['played',
 '2PM',
 '2PA',
 'Dunks made',
 'Dunks attempted',
 'Layups made',
 'Layups attempted',
 'Floaters made',
 'Floaters attempted',
 'Hook Shots made',
 'Hook Shots attempted',
 'Jumpshots made',
 'Jumpshots attempted',
 'Pullups made',
 'Pullups attempted',
 'Turnarounds made',
 'Turnarounds attempted',
 'Step Backs made',
 'Step Backs attempted',
 'Jumpshots assisted on',
 '1 made',
 '2-1 made',
 '2-2 made',
 '2-3 made',
 '3-1 made',
 '3-2 made',
 '3-3 made',
 '4-1 made',
 '4-2 made',
 '4-3 made',
 '1 attempted',
 '2-1 attempted',
 '2-2 attempted',
 '2-3 attempted',
 '3-1 attempted',
 '3-2 attempted',
 '3-3 attempted',
 '4-1 attempted',
 '4-2 attempted',
 '4-3 attempted']

In [27]:
two_pointers_df.insert(4, '2P%', two_pointers_df['2PM']/two_pointers_df['2PA'])
two_pointers_df.insert(7, 'Dunk FG%', two_pointers_df['Dunks made']/two_pointers_df['Dunks attempted'])
two_pointers_df.insert(10, 'Layup FG%', two_pointers_df['Layups made']/two_pointers_df['Layups attempted'])
two_pointers_df.insert(13, 'Floater FG%', two_pointers_df['Floaters made']/two_pointers_df['Floaters attempted'])
two_pointers_df.insert(16, 'Hook Shot FG%', two_pointers_df['Hook Shots made']/two_pointers_df['Hook Shots attempted'])
two_pointers_df.insert(19, 'Jumpshot FG%', two_pointers_df['Jumpshots made']/two_pointers_df['Jumpshots attempted'])
two_pointers_df.insert(22, 'Pullup FG%', two_pointers_df['Pullups made']/two_pointers_df['Pullups attempted'])
two_pointers_df.insert(25, 'Turnaround FG%', two_pointers_df['Turnarounds made']/two_pointers_df['Turnarounds attempted'])
two_pointers_df.insert(28, 'Step Back FG%', two_pointers_df['Step Backs made']/two_pointers_df['Step Backs attempted'])
two_pointers_df.insert(30, 'Jumpshots assisted%', two_pointers_df['Jumpshots assisted on']/two_pointers_df['2PM'])
two_pointers_df['zone 1 FG%'] = two_pointers_df['1 made']/two_pointers_df['1 attempted']
two_pointers_df['zone 2-1 FG%'] = two_pointers_df['2-1 made']/two_pointers_df['2-1 attempted']
two_pointers_df['zone 2-2 FG%'] = two_pointers_df['2-2 made']/two_pointers_df['2-2 attempted']
two_pointers_df['zone 2-3 FG%'] = two_pointers_df['2-3 made']/two_pointers_df['2-3 attempted']
two_pointers_df['zone 3-1 FG%'] = two_pointers_df['3-1 made']/two_pointers_df['3-1 attempted']
two_pointers_df['zone 3-2 FG%'] = two_pointers_df['3-2 made']/two_pointers_df['3-2 attempted']
two_pointers_df['zone 3-3 FG%'] = two_pointers_df['3-3 made']/two_pointers_df['3-3 attempted']
two_pointers_df['zone 4-1 FG%'] = two_pointers_df['4-1 made']/two_pointers_df['4-1 attempted']
two_pointers_df['zone 4-2 FG%'] = two_pointers_df['4-2 made']/two_pointers_df['4-2 attempted']
two_pointers_df['zone 4-3 FG%'] = two_pointers_df['4-3 made']/two_pointers_df['4-3 attempted']

In [28]:
two_pointers_df['2PM'] = two_pointers_df['2PM']/two_pointers_df.played
two_pointers_df['2PA'] = two_pointers_df['2PA']/two_pointers_df.played
two_pointers_df['Dunks made'] = two_pointers_df['Dunks made']/two_pointers_df.played
two_pointers_df['Dunks attempted'] = two_pointers_df['Dunks attempted']/two_pointers_df.played
two_pointers_df['Layups made'] = two_pointers_df['Layups made']/two_pointers_df.played
two_pointers_df['Layups attempted'] = two_pointers_df['Layups attempted']/two_pointers_df.played
two_pointers_df['Floaters made'] = two_pointers_df['Floaters made']/two_pointers_df.played
two_pointers_df['Floaters attempted'] = two_pointers_df['Floaters attempted']/two_pointers_df.played
two_pointers_df['Hook Shots made'] = two_pointers_df['Hook Shots made']/two_pointers_df.played
two_pointers_df['Hook Shots attempted'] = two_pointers_df['Hook Shots attempted']/two_pointers_df.played
two_pointers_df['Jumpshots made'] = two_pointers_df['Jumpshots made']/two_pointers_df.played
two_pointers_df['Jumpshots attempted'] = two_pointers_df['Jumpshots attempted']/two_pointers_df.played
two_pointers_df['Pullups made'] = two_pointers_df['Pullups made']/two_pointers_df.played
two_pointers_df['Pullups attempted'] = two_pointers_df['Pullups attempted']/two_pointers_df.played
two_pointers_df['Turnarounds made'] = two_pointers_df['Turnarounds made']/two_pointers_df.played
two_pointers_df['Turnarounds attempted'] = two_pointers_df['Turnarounds attempted']/two_pointers_df.played
two_pointers_df['Step Backs made'] = two_pointers_df['Step Backs made']/two_pointers_df.played
two_pointers_df['Step Backs attempted'] = two_pointers_df['Step Backs attempted']/two_pointers_df.played
two_pointers_df['1 made'] = two_pointers_df['1 made']/two_pointers_df.played
two_pointers_df['1 attempted'] = two_pointers_df['1 attempted']/two_pointers_df.played
two_pointers_df['2-1 made'] = two_pointers_df['2-1 made']/two_pointers_df.played
two_pointers_df['2-1 attempted'] = two_pointers_df['2-1 attempted']/two_pointers_df.played
two_pointers_df['2-2 made'] = two_pointers_df['2-2 made']/two_pointers_df.played
two_pointers_df['2-2 attempted'] = two_pointers_df['2-2 attempted']/two_pointers_df.played
two_pointers_df['2-3 made'] = two_pointers_df['2-3 made']/two_pointers_df.played
two_pointers_df['2-3 attempted'] = two_pointers_df['2-3 attempted']/two_pointers_df.played
two_pointers_df['3-1 made'] = two_pointers_df['3-1 made']/two_pointers_df.played
two_pointers_df['3-1 attempted'] = two_pointers_df['3-1 attempted']/two_pointers_df.played
two_pointers_df['3-2 made'] = two_pointers_df['3-2 made']/two_pointers_df.played
two_pointers_df['3-2 attempted'] = two_pointers_df['3-2 attempted']/two_pointers_df.played
two_pointers_df['3-3 made'] = two_pointers_df['3-3 made']/two_pointers_df.played
two_pointers_df['3-3 attempted'] = two_pointers_df['3-3 attempted']/two_pointers_df.played
two_pointers_df['4-1 made'] = two_pointers_df['4-1 made']/two_pointers_df.played
two_pointers_df['4-1 attempted'] = two_pointers_df['4-1 attempted']/two_pointers_df.played
two_pointers_df['4-2 made'] = two_pointers_df['4-2 made']/two_pointers_df.played
two_pointers_df['4-2 attempted'] = two_pointers_df['4-2 attempted']/two_pointers_df.played
two_pointers_df['4-3 made'] = two_pointers_df['4-3 made']/two_pointers_df.played
two_pointers_df['4-3 attempted'] = two_pointers_df['4-3 attempted']/two_pointers_df.played

In [29]:
two_pointers_df = two_pointers_df.add_prefix("Last_season ")
two_pointers_df = two_pointers_df.rename(columns={'Last_season player_id':'player_id'})
two_pointers_df

,player_id,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_season Hook Shots made,Last_season Hook Shots attempted,Last_season Hook Shot FG%,Last_season Jumpshots made,Last_season Jumpshots attempted,Last_season Jumpshot FG%,Last_season Pullups made,Last_season Pullups attempted,Last_season Pullup FG%,Last_season Turnarounds made,Last_season Turnarounds attempted,Last_season Turnaround FG%,Last_season Step Backs made,Last_season Step Backs attempted,Last_season Step Back FG%,Last_season Jumpshots assisted on,Last_season Jumpshots assisted%,Last_season 1 made,Last_season 2-1 made,Last_season 2-2 made,Last_season 2-3 made,Last_season 3-1 made,Last_season 3-2 made,Last_season 3-3 made,Last_season 4-1 made,Last_season 4-2 made,Last_season 4-3 made,Last_season 1 attempted,Last_season 2-1 attempted,Last_season 2-2 attempted,Last_season 2-3 attempted,Last_season 3-1 attempted,Last_season 3-2 attempted,Last_season 3-3 attempted,Last_season 4-1 attempted,Last_season 4-2 attempted,Last_season 4-3 attempted,Last_season zone 1 FG%,Last_season zone 2-1 FG%,Last_season zone 2-2 FG%,Last_season zone 2-3 FG%,Last_season zone 3-1 FG%,Last_season zone 3-2 FG%,Last_season zone 3-3 FG%,Last_season zone 4-1 FG%,Last_season zone 4-2 FG%,Last_season zone 4-3 FG%
0,618da14c4db298feb94be0d1,24,2.666667,5.875000,0.453901,0.041667,0.041667,1.0,1.666667,3.250000,0.512821,0.125000,0.291667,0.428571,0.041667,0.083333,0.500000,0.083333,0.750000,0.111111,0.0,0.041667,0.0,0.208333,0.666667,0.3125,0.083333,0.083333,1.0,0.0,0.000000,1.833333,0.166667,0.166667,0.083333,0.083333,0.125000,0.041667,0.041667,0.125000,0.0,3.333333,0.541667,0.625000,0.166667,0.208333,0.458333,0.125000,0.041667,0.375000,0.000000,0.550000,0.307692,0.266667,0.500000,0.400000,0.272727,0.333333,1.0,0.333333,NaN
1,618da14c4db298feb94be0d2,24,3.708333,7.083333,0.523529,0.083333,0.083333,1.0,2.333333,4.500000,0.518519,0.000000,0.041667,0.000000,0.500000,0.833333,0.600000,0.083333,0.583333,0.142857,0.0,0.000000,NaN,0.083333,0.333333,0.2500,0.000000,0.000000,NaN,0.0,0.000000,2.958333,0.166667,0.416667,0.041667,0.041667,0.041667,0.041667,0.000000,0.000000,0.0,4.833333,0.500000,1.041667,0.291667,0.125000,0.166667,0.041667,0.000000,0.041667,0.041667,0.612069,0.333333,0.400000,0.142857,0.333333,0.250000,1.000000,NaN,0.000000,0.0
2,618da14c4db298feb94be0d3,23,0.478261,0.913043,0.523810,0.000000,0.000000,NaN,0.304348,0.521739,0.583333,0.043478,0.043478,1.000000,0.000000,0.043478,0.000000,0.086957,0.130435,0.666667,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,1.0,0.090909,0.260870,0.086957,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.0,0.521739,0.086957,0.086957,0.000000,0.000000,0.130435,0.000000,0.000000,0.086957,0.000000,0.500000,1.000000,0.500000,NaN,NaN,0.333333,NaN,NaN,0.500000,NaN
3,618da14c4db298feb94be0d4,24,2.666667,5.041667,0.528926,0.000000,0.000000,NaN,1.208333,2.375000,0.508772,0.125000,0.166667,0.750000,0.625000,0.916667,0.681818,0.458333,1.000000,0.458333,0.0,0.000000,NaN,0.125000,0.416667,0.3000,0.000000,0.000000,NaN,2.0,0.031250,1.208333,0.333333,0.541667,0.041667,0.041667,0.166667,0.041667,0.041667,0.250000,0.0,2.125000,0.500000,0.875000,0.125000,0.250000,0.583333,0.041667,0.041667,0.500000,0.000000,0.568627,0.666667,0.619048,0.333333,0.166667,0.285714,1.000000,1.0,0.500000,NaN
4,618da14c4db298feb94be0d5,22,1.727273,4.181818,0.413043,0.000000,0.000000,NaN,1.545455,2.818182,0.548387,0.000000,0.136364,0.000000,0.000000,0.090909,0.000000,0.090909,0.818182,0.111111,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.000000,1.227273,0.090909,0.227273,0.090909,0.000000,0.045455,0.045455,0.000000,0.000000,0.0,2.409091,0.227273,0.545455,0.272727,0.045455,0.318182,0.227273,0.045455,0.090909,0.0

In [30]:
#write the data to a pickle file
two_pointers_df.to_pickle("Last_season_2P_features_df.pickle")

Feature creation for 3 pointers

In [31]:
#Filter on 3 point shots
Threes_df = shots_df[shots_df.Type=='Three']
three_pointers_df = Threes_df.groupby(['player_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'3PM', 'count':'3PA'})
three_pointers_df

,player_id,3PM,3PA
0,618bf7f2815098ca6660f331,0,2
1,618bf7f2815098ca6660f332,37,104
2,618bf7f3815098ca6660f333,39,109
3,618bf7f3815098ca6660f334,52,152
4,618bf7f3815098ca6660f335,0,3
...,...,...,...
4108,625648232c06a1d63387153e,1,1
4109,625648232c06a1d63387153f,0,1
4110,6256496b2c06a1d633876187,1,2
4111,62564bb12c06a1d63387f6b5,0,1


In [32]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
three_pointers_df = pd.merge(player_team[['player_id','played']], three_pointers_df, how= 'left', on=['player_id']).fillna(0)
three_pointers_df

,player_id,played,3PM,3PA
0,618da14c4db298feb94be0d1,24,28.0,88.0
1,618da14c4db298feb94be0d2,24,0.0,1.0
2,618da14c4db298feb94be0d3,23,19.0,60.0
3,618da14c4db298feb94be0d4,24,24.0,73.0
4,618da14c4db298feb94be0d5,22,19.0,59.0
...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,1.0,5.0
4395,6255a71bed9cbb2fe0fe5d07,18,2.0,17.0
4396,6255a71bed9cbb2fe0fe5d08,7,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0


In [33]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Jumpshot', 'Pullup', 'Step Back']:
    temp = Threes_df[Threes_df[stat]==True]
    temp_df = temp.groupby(['player_id']).Result.agg(
        ['sum','count']).reset_index().rename(columns={'sum': '3P '+stat+'s made', 'count': '3P '+stat+'s attempted'})
    three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)
three_pointers_df

,player_id,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted
0,618da14c4db298feb94be0d1,24,28.0,88.0,28.0,88.0,0.0,0.0,0.0,0.0
1,618da14c4db298feb94be0d2,24,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,618da14c4db298feb94be0d3,23,19.0,60.0,17.0,58.0,0.0,0.0,1.0,1.0
3,618da14c4db298feb94be0d4,24,24.0,73.0,23.0,72.0,0.0,0.0,1.0,1.0
4,618da14c4db298feb94be0d5,22,19.0,59.0,19.0,59.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,1.0,5.0,1.0,5.0,0.0,0.0,0.0,0.0
4395,6255a71bed9cbb2fe0fe5d07,18,2.0,17.0,2.0,17.0,0.0,0.0,0.0,0.0
4396,6255a71bed9cbb2fe0fe5d08,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
#Calculated percentage of made 3p jumpshots that were assisted
temp_df = Threes_df[(Threes_df['Jumpshot']==True)&(Threes_df['Result']==1)].groupby(
    ['player_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': '3P Jumpshots assisted on'})
temp_df

,player_id,3P Jumpshots assisted on
0,618bf7f2815098ca6660f332,31
1,618bf7f3815098ca6660f333,32
2,618bf7f3815098ca6660f334,36
3,618bf7f3815098ca6660f336,25
4,618bf7f3815098ca6660f337,1
...,...,...
3488,625632e02c06a1d633819acb,0
3489,625635df2c06a1d6338268a5,1
3490,625639d02c06a1d63383923e,1
3491,625648232c06a1d63387153e,1


In [35]:
three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id']).fillna(0)
three_pointers_df

,player_id,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted,3P Jumpshots assisted on
0,618da14c4db298feb94be0d1,24,28.0,88.0,28.0,88.0,0.0,0.0,0.0,0.0,27.0
1,618da14c4db298feb94be0d2,24,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,618da14c4db298feb94be0d3,23,19.0,60.0,17.0,58.0,0.0,0.0,1.0,1.0,17.0
3,618da14c4db298feb94be0d4,24,24.0,73.0,23.0,72.0,0.0,0.0,1.0,1.0,16.0
4,618da14c4db298feb94be0d5,22,19.0,59.0,19.0,59.0,0.0,0.0,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,1.0,5.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0
4395,6255a71bed9cbb2fe0fe5d07,18,2.0,17.0,2.0,17.0,0.0,0.0,0.0,0.0,2.0
4396,6255a71bed9cbb2fe0fe5d08,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#3P% per zone for each player
Zones3 = Threes_df.groupby(['player_id','Zone']).Result.agg(['sum','count']).reset_index()
Zones3

,player_id,Zone,sum,count
0,618bf7f2815098ca6660f331,6-2,0,2
1,618bf7f2815098ca6660f332,5-1,4,11
2,618bf7f2815098ca6660f332,5-2,1,15
3,618bf7f2815098ca6660f332,6-1,2,9
4,618bf7f2815098ca6660f332,6-2,11,24
...,...,...,...,...
24197,625648232c06a1d63387153f,6-3,0,1
24198,6256496b2c06a1d633876187,5-1,0,1
24199,6256496b2c06a1d633876187,7-1,1,1
24200,62564bb12c06a1d63387f6b5,6-1,0,1


In [37]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones3.pivot(index=['player_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])##remove leading space
pivot_zones

,player_id,5-1 made,5-2 made,6-1 made,6-2 made,6-3 made,7-1 made,7-2 made,7-3 made,5-1 attempted,5-2 attempted,6-1 attempted,6-2 attempted,6-3 attempted,7-1 attempted,7-2 attempted,7-3 attempted
0,618bf7f2815098ca6660f331,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
1,618bf7f2815098ca6660f332,4.0,1.0,2.0,11.0,4.0,4.0,6.0,5.0,11.0,15.0,9.0,24.0,12.0,4.0,22.0,7.0
2,618bf7f3815098ca6660f333,2.0,4.0,3.0,15.0,4.0,0.0,9.0,2.0,2.0,5.0,11.0,44.0,10.0,5.0,26.0,6.0
3,618bf7f3815098ca6660f334,3.0,1.0,5.0,11.0,6.0,2.0,21.0,3.0,7.0,5.0,17.0,43.0,19.0,5.0,51.0,5.0
4,618bf7f3815098ca6660f335,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108,625648232c06a1d63387153e,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4109,625648232c06a1d63387153f,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4110,6256496b2c06a1d633876187,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4111,62564bb12c06a1d63387f6b5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [38]:
three_pointers_df = pd.merge(three_pointers_df, pivot_zones, how= 'left', on=['player_id']).fillna(0)
three_pointers_df

,player_id,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted,3P Jumpshots assisted on,5-1 made,5-2 made,6-1 made,6-2 made,6-3 made,7-1 made,7-2 made,7-3 made,5-1 attempted,5-2 attempted,6-1 attempted,6-2 attempted,6-3 attempted,7-1 attempted,7-2 attempted,7-3 attempted
0,618da14c4db298feb94be0d1,24,28.0,88.0,28.0,88.0,0.0,0.0,0.0,0.0,27.0,3.0,1.0,4.0,5.0,7.0,0.0,6.0,2.0,9.0,14.0,13.0,13.0,18.0,1.0,17.0,3.0
1,618da14c4db298feb94be0d2,24,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,618da14c4db298feb94be0d3,23,19.0,60.0,17.0,58.0,0.0,0.0,1.0,1.0,17.0,2.0,0.0,6.0,3.0,1.0,0.0,7.0,0.0,6.0,3.0,13.0,15.0,6.0,3.0,14.0,0.0
3,618da14c4db298feb94be0d4,24,24.0,73.0,23.0,72.0,0.0,0.0,1.0,1.0,16.0,4.0,1.0,5.0,7.0,1.0,2.0,3.0,1.0,5.0,2.0,5.0,22.0,6.0,2.0,24.0,7.0
4,618da14c4db298feb94be0d5,22,19.0,59.0,19.0,59.0,0.0,0.0,0.0,0.0,18.0,1.0,3.0,3.0,4.0,2.0,1.0,4.0,1.0,3.0,5.0,10.0,13.0,12.0,1.0,12.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,1.0,5.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0
4395,6255a71bed9cbb2fe0fe5d07,18,2.0,17.0,2.0,17.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,5.0,3.0,2.0,1.0,1.0,2.0
4396,6255a71bed9cbb2fe0fe5d08,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,6255cd5d2c06a1d633649449,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [39]:
three_pointers_df.columns[1:].to_list()

['played',
 '3PM',
 '3PA',
 '3P Jumpshots made',
 '3P Jumpshots attempted',
 '3P Pullups made',
 '3P Pullups attempted',
 '3P Step Backs made',
 '3P Step Backs attempted',
 '3P Jumpshots assisted on',
 '5-1 made',
 '5-2 made',
 '6-1 made',
 '6-2 made',
 '6-3 made',
 '7-1 made',
 '7-2 made',
 '7-3 made',
 '5-1 attempted',
 '5-2 attempted',
 '6-1 attempted',
 '6-2 attempted',
 '6-3 attempted',
 '7-1 attempted',
 '7-2 attempted',
 '7-3 attempted']

In [40]:
#For each dataframe and each shot/zone type compute the respective percentages
three_pointers_df.insert(4, '3P%', three_pointers_df['3PM']/three_pointers_df['3PA'])
three_pointers_df.insert(7, 'Jumpshots 3P%', three_pointers_df['3P Jumpshots made']/three_pointers_df['3P Jumpshots attempted'])
three_pointers_df.insert(10, 'Pullups 3P%', three_pointers_df['3P Pullups made']/three_pointers_df['3P Pullups attempted'])
three_pointers_df.insert(13, 'Step Backs 3P%', three_pointers_df['3P Step Backs made']/three_pointers_df['3P Step Backs attempted'])
three_pointers_df.insert(16, '3P Jumpshots assisted%', three_pointers_df['3P Jumpshots assisted on']/three_pointers_df['3PM'])
three_pointers_df['zone 5-1 FG%'] = three_pointers_df['5-1 made']/three_pointers_df['5-1 attempted']
three_pointers_df['zone 5-2 FG%'] = three_pointers_df['5-2 made']/three_pointers_df['5-2 attempted']
three_pointers_df['zone 6-1 FG%'] = three_pointers_df['6-1 made']/three_pointers_df['6-1 attempted']
three_pointers_df['zone 6-2 FG%'] = three_pointers_df['6-2 made']/three_pointers_df['6-2 attempted']
three_pointers_df['zone 6-3 FG%'] = three_pointers_df['6-3 made']/three_pointers_df['6-3 attempted']
three_pointers_df['zone 7-1 FG%'] = three_pointers_df['7-1 made']/three_pointers_df['7-1 attempted']
three_pointers_df['zone 7-2 FG%'] = three_pointers_df['7-2 made']/three_pointers_df['7-2 attempted']
three_pointers_df['zone 7-3 FG%'] = three_pointers_df['7-3 made']/three_pointers_df['7-3 attempted']

In [41]:
three_pointers_df['3PM'] = three_pointers_df['3PM']/three_pointers_df.played
three_pointers_df['3PA'] = three_pointers_df['3PA']/three_pointers_df.played
three_pointers_df['3P Jumpshots made'] = three_pointers_df['3P Jumpshots made']/three_pointers_df.played
three_pointers_df['3P Jumpshots attempted'] = three_pointers_df['3P Jumpshots attempted']/three_pointers_df.played
three_pointers_df['3P Pullups made'] = three_pointers_df['3P Pullups made']/three_pointers_df.played
three_pointers_df['3P Pullups attempted'] = three_pointers_df['3P Pullups attempted']/three_pointers_df.played
three_pointers_df['3P Step Backs made'] = three_pointers_df['3P Step Backs made']/three_pointers_df.played
three_pointers_df['3P Step Backs attempted'] = three_pointers_df['3P Step Backs attempted']/three_pointers_df.played
three_pointers_df['5-1 made'] = three_pointers_df['5-1 made']/three_pointers_df.played
three_pointers_df['5-1 attempted'] = three_pointers_df['5-1 attempted']/three_pointers_df.played
three_pointers_df['5-2 made'] = three_pointers_df['5-2 made']/three_pointers_df.played
three_pointers_df['5-2 attempted'] = three_pointers_df['5-2 attempted']/three_pointers_df.played
three_pointers_df['6-1 made'] = three_pointers_df['6-1 made']/three_pointers_df.played
three_pointers_df['6-1 attempted'] = three_pointers_df['6-1 attempted']/three_pointers_df.played
three_pointers_df['6-2 made'] = three_pointers_df['6-2 made']/three_pointers_df.played
three_pointers_df['6-2 attempted'] = three_pointers_df['6-2 attempted']/three_pointers_df.played
three_pointers_df['6-3 made'] = three_pointers_df['6-3 made']/three_pointers_df.played
three_pointers_df['6-3 attempted'] = three_pointers_df['6-3 attempted']/three_pointers_df.played
three_pointers_df['7-1 made'] = three_pointers_df['7-1 made']/three_pointers_df.played
three_pointers_df['7-1 attempted'] = three_pointers_df['7-1 attempted']/three_pointers_df.played
three_pointers_df['7-2 made'] = three_pointers_df['7-2 made']/three_pointers_df.played
three_pointers_df['7-2 attempted'] = three_pointers_df['7-2 attempted']/three_pointers_df.played
three_pointers_df['7-3 made'] = three_pointers_df['7-3 made']/three_pointers_df.played
three_pointers_df['7-3 attempted'] = three_pointers_df['7-3 attempted']/three_pointers_df.played

In [42]:
three_pointers_df = three_pointers_df.add_prefix("Last_season ")
three_pointers_df = three_pointers_df.rename(columns={'Last_season player_id':'player_id'})
three_pointers_df

,player_id,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 5-1 made,Last_season 3P Jumpshots assisted%,Last_season 5-2 made,Last_season 6-1 made,Last_season 6-2 made,Last_season 6-3 made,Last_season 7-1 made,Last_season 7-2 made,Last_season 7-3 made,Last_season 5-1 attempted,Last_season 5-2 attempted,Last_season 6-1 attempted,Last_season 6-2 attempted,Last_season 6-3 attempted,Last_season 7-1 attempted,Last_season 7-2 attempted,Last_season 7-3 attempted,Last_season zone 5-1 FG%,Last_season zone 5-2 FG%,Last_season zone 6-1 FG%,Last_season zone 6-2 FG%,Last_season zone 6-3 FG%,Last_season zone 7-1 FG%,Last_season zone 7-2 FG%,Last_season zone 7-3 FG%
0,618da14c4db298feb94be0d1,24,1.166667,3.666667,0.318182,1.166667,3.666667,0.318182,0.0,0.0,NaN,0.000000,0.000000,NaN,27.0,0.125000,0.964286,0.041667,0.166667,0.208333,0.291667,0.000000,0.250000,0.083333,0.375000,0.583333,0.541667,0.541667,0.750000,0.041667,0.708333,0.125000,0.333333,0.071429,0.307692,0.384615,0.388889,0.0,0.352941,0.666667
1,618da14c4db298feb94be0d2,24,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
2,618da14c4db298feb94be0d3,23,0.826087,2.608696,0.316667,0.739130,2.521739,0.293103,0.0,0.0,NaN,0.043478,0.043478,1.0,17.0,0.086957,0.894737,0.000000,0.260870,0.130435,0.043478,0.000000,0.304348,0.000000,0.260870,0.130435,0.565217,0.652174,0.260870,0.130435,0.608696,0.000000,0.333333,0.000000,0.461538,0.200000,0.166667,0.0,0.500000,NaN
3,618da14c4db298feb94be0d4,24,1.000000,3.041667,0.328767,0.958333,3.000000,0.319444,0.0,0.0,NaN,0.041667,0.041667,1.0,16.0,0.166667,0.666667,0.041667,0.208333,0.291667,0.041667,0.083333,0.125000,0.041667,0.208333,0.083333,0.208333,0.916667,0.250000,0.083333,1.000000,0.291667,0.800000,0.500000,1.000000,0.318182,0.166667,1.0,0.125000,0.142857
4,618da14c4db298feb94be0d5,22,0.863636,2.681818,0.322034,0.863636,2.681818,0.322034,0.0,0.0,NaN,0.000000,0.000000,NaN,18.0,0.045455,0.947368,0.136364,0.136364,0.181818,0.090909,0.045455,0.181818,0.045455,0.136364,0.227273,0.454545,0.590909,0.545455,0.045455,0.545455,0.136364,0.333333,0.600000,0.300000,0.307692,0.166667,1.0,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,0.083333,0.416667,0.200000,0.083333,0.416667,0.200000,0.0,0.0,NaN,0.000000,0.000000,NaN,1.0,0.083333,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.083333,0.000000,0.000000,0.500000,NaN,NaN,0.000000,NaN,0.0,NaN,NaN
4395,6255a71bed9cbb2fe0fe5d07,18,0.111111,0.944444,0.117647,0.111111,0.944444,0.117647,0.0,0.0,NaN,0.000000,0.000000,NaN,2.0,0.055556,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.111111,0.055556,0.277778,0.166667,0.111111,0.055556,0.055556,0.111111,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000
4396,6255a71bed9cbb2fe0fe5d08,7,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4397,6255cd5d2c06a1d633649449,2,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [43]:
#write the data to a pickle file
three_pointers_df.to_pickle("Last_season_3P_features_df.pickle")